<a href="https://colab.research.google.com/github/Maruf346/AI-ML-with-python/blob/main/Thesis%20MADDPG%20Dataset%20for%20paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import random

# --- 1. CONFIGURATION ---
NUM_ROWS = 1000
NUM_DEVICES = 4
OUTPUT_FILENAME = 'PRiFARED_maddpg_dataset_1000.csv'

# Define the precise ranges for min-max scaling later
RANGES = {
    'C_cpu': (10, 100),
    'C_data': (5, 50),
    'T_max': (100, 500),
    'Rn': (0, 9), # Integer range
    'Battery Level (%)': (1, 100),
    'Q_k': (0, 50),
    'F_max': (1, 5),
    'B_max': (5, 30),
    'P_max': (1, 6),
    'x_hat_alloc': (0, 1), # Allocations are fractions [0, 1]
    # Placeholders for results
    'Latency': (50, 600),
    'Energy': (0.5, 12.0)
}

# Define categorical mappings
TASK_CLASSES = {0: '0 (Normal)', 1: '1 (Medium)', 2: '2 (Urgent)'}
USER_CONTRACTS = {0: '0 (Normal)', 1: '1 (Premium)'}

# --- 2. HELPER FUNCTIONS ---

def generate_continuous(min_val, max_val, num_samples):
    """Generates random continuous values within a specified range."""
    return np.random.uniform(min_val, max_val, num_samples)

def generate_integer(min_val, max_val, num_samples):
    """Generates random integer values within a specified range."""
    return np.random.randint(min_val, max_val + 1, num_samples)

def generate_vector(min_val, max_val, num_samples, vector_size):
    """Generates vectors (lists of floats) for device parameters."""
    data = []
    for _ in range(num_samples):
        # Generate random values for the vector
        vector = np.random.uniform(min_val, max_val, vector_size)
        # Store as a string representation to match the original markdown format
        data.append('$$' + ', '.join(f'{x:.2f}' for x in vector) + '$$')
    return data

def generate_allocation_vector(num_samples, vector_size):
    """Generates a normalized allocation vector (sums to 1.0)."""
    data = []
    for _ in range(num_samples):
        # Generate raw random values
        raw_allocs = np.random.uniform(0.01, 1.0, vector_size)
        # Normalize so they sum to 1.0 (to respect resource limits)
        normalized_allocs = raw_allocs / np.sum(raw_allocs)
        # Store as a string
        data.append('$$' + ', '.join(f'{x:.3f}' for x in normalized_allocs) + '$$')
    return data

# --- 3. DATA GENERATION ---

data = pd.DataFrame()

# 1. Task Feature Inputs
data['Task ID'] = np.arange(1, NUM_ROWS + 1)
data['C_cpu (GC) [10-100]'] = generate_continuous(*RANGES['C_cpu'], NUM_ROWS)
data['C_data (Mbits) [5-50]'] = generate_continuous(*RANGES['C_data'], NUM_ROWS)
data['T_max (ms) [100-500]'] = generate_continuous(*RANGES['T_max'], NUM_ROWS)
data['Rn (Retry) [0-9]'] = generate_integer(*RANGES['Rn'], NUM_ROWS)
data['Battery Level (%) [1-100]'] = generate_continuous(*RANGES['Battery Level (%)'], NUM_ROWS)

# 2. Categorical Inputs (using mappings)
task_classes_raw = generate_integer(0, 2, NUM_ROWS)
user_contracts_raw = generate_integer(0, 1, NUM_ROWS)
data['Task_Service_Class [0/1/2]'] = [TASK_CLASSES[c] for c in task_classes_raw]
data['User_Contract_Type [0/1]'] = [USER_CONTRACTS[c] for c in user_contracts_raw]

# 3. Environment Vector Inputs (4 devices)
data['Q_k (Queue) [0-50]'] = generate_vector(*RANGES['Q_k'], NUM_ROWS, NUM_DEVICES)
data['F_max (GHz) [1-5]'] = generate_vector(*RANGES['F_max'], NUM_ROWS, NUM_DEVICES)
data['B_max (MHz) [5-30]'] = generate_vector(*RANGES['B_max'], NUM_ROWS, NUM_DEVICES)
data['P_max (W) [1-6]'] = generate_vector(*RANGES['P_max'], NUM_ROWS, NUM_DEVICES)

# 4. Action Placeholder Outputs (Normalized allocations)
data['x_hat_cpu (Alloc) [0-1]'] = generate_allocation_vector(NUM_ROWS, NUM_DEVICES)
data['x_hat_bw (Alloc) [0-1]'] = generate_allocation_vector(NUM_ROWS, NUM_DEVICES)
data['x_hat_pow (Alloc) [0-1]'] = generate_allocation_vector(NUM_ROWS, NUM_DEVICES)

# 5. Result Placeholder Outputs (These are synthetic results for a dataset)
data['Latency (ms) [Result]'] = generate_continuous(*RANGES['Latency'], NUM_ROWS)
data['Energy (J) [Result]'] = generate_continuous(*RANGES['Energy'], NUM_ROWS)
# Success is 1 if Latency is less than T_max, 0 otherwise (approximated for this placeholder data)
data['Success (0/1) [Result]'] = (data['Latency (ms) [Result]'] < data['T_max (ms) [100-500]']).astype(int)

# --- 4. OUTPUT AND DISPLAY ---

# Save the dataset to a CSV file (typical for Colab or training environments)
data.to_csv(OUTPUT_FILENAME, index=False)

print(f"Successfully generated {NUM_ROWS} rows of data.")
print(f"Data saved to {OUTPUT_FILENAME}")
print("\nFirst 5 rows:")
print(data.head())
print(f"\nDataFrame shape: {data.shape}")


Successfully generated 1000 rows of data.
Data saved to priafred_maddpg_dataset_1000.csv

First 5 rows:
   Task ID  C_cpu (GC) [10-100]  C_data (Mbits) [5-50]  T_max (ms) [100-500]  \
0        1            22.464309              46.424205            189.116394   
1        2            89.953347              24.711431            481.847970   
2        3            67.874361              37.458598            316.745533   
3        4            73.835563              30.903319            117.997995   
4        5            37.323252               7.020714            337.546378   

   Rn (Retry) [0-5]  Battery Level (%) [20-100] Task_Service_Class [0/1/2]  \
0                 5                   87.725559                 1 (Medium)   
1                 2                   51.017357                 0 (Normal)   
2                 3                   43.270520                 0 (Normal)   
3                 1                   28.697589                 1 (Medium)   
4                 5      

In [ ]:
display(data)

,Task ID,C_cpu (GC) [10-100],C_data (Mbits) [5-50],T_max (ms) [100-500],Rn (Retry) [0-5],Battery Level (%) [20-100],Task_Service_Class [0/1/2],User_Contract_Type [0/1],Q_k (Queue) [0-50],F_max (GHz) [1-5],B_max (MHz) [5-30],P_max (W) [1-6],x_hat_cpu (Alloc) [0-1],x_hat_bw (Alloc) [0-1],x_hat_pow (Alloc) [0-1],Latency (ms) [Result],Energy (J) [Result],Success (0/1) [Result]
0,1,22.464309,46.424205,189.116394,5,87.725559,1 (Medium),1 (Premium),"$$26.97, 39.76, 29.55, 25.35$$","$$3.21, 4.89, 4.59, 3.87$$","$$14.53, 23.87, 5.72, 26.55$$","$$4.65, 3.41, 2.66, 4.11$$","$$0.466, 0.292, 0.019, 0.222$$","$$0.304, 0.366, 0.221, 0.109$$","$$0.277, 0.556, 0.090, 0.078$$",252.732361,4.059276,0
1,2,89.953347,24.711431,481.847970,2,51.017357,0 (Normal),1 (Premium),"$$43.29, 23.16, 25.88, 41.73$$","$$1.69, 2.41, 4.35, 1.45$$","$$9.15, 29.56, 25.27, 15.55$$","$$4.39, 4.57, 1.43, 3.77$$","$$0.373, 0.240, 0.046, 0.340$$","$$0.043, 0.655, 0.091, 0.211$$","$$0.030, 0.263, 0.229, 0.478$$",345.950070,1.100554,1
2,3,67.874361,37.458598,316.745533,3,43.270520,0 (Normal),1 (Premium),"$$29.18, 37.99, 35.55, 4.44$$","$$3.44, 2.93, 3.14, 3.36$$","$$28.58, 25.81, 13.69, 24.88$$","$$1.64, 4.17, 5.24, 5.31$$","$$0.482, 0.278, 0.052, 0.187$$","$$0.321, 0.310, 0.086, 0.283$$","$$0.211, 0.120, 0.125, 0.543$$",583.598772,3.284807,0
3,4,73.835563,30.903319,117.997995,1,28.697589,1 (Medium),0 (Normal),"$$8.02, 41.35, 41.43, 43.55$$","$$2.89, 2.31, 1.34, 1.65$$","$$29.18, 11.84, 24.98, 19.78$$","$$1.77, 3.59, 1.16, 5.94$$","$$0.060, 0.057, 0.607, 0.276$$","$$0.333, 0.329, 0.095, 0.243$$","$$0.476, 0.295, 0.157, 0.072$$",106.046424,10.172869,1
4,5,37.323252,7.020714,337.546378,5,89.993548,0 (Normal),0 (Normal),"$$10.23, 12.33, 6.94, 4.00$$","$$2.45, 4.55, 4.94, 1.46$$","$$18.17, 17.38, 6.10, 24.92$$","$$1.28, 3.43, 3.33, 3.37$$","$$0.050, 0.337, 0.136, 0.477$$","$$0.015, 0.173, 0.375, 0.437$$","$$0.290, 0.534, 0.049, 0.127$$",489.415111,3.666849,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,49.290919,32.123185,446.750134,0,96.002703,1 (Medium),1 (Premium),"$$22.08, 33.49, 38.60, 49.76$$","$$3.50, 2.81, 1.93, 1.63$$","$$18.08, 27.77, 25.62, 11.01$$","$$3.42, 3.97, 3.23, 5.57$$","$$0.583, 0.045, 0.354, 0.019$$","$$0.122, 0.375, 0.238, 0.264$$","$$0.195, 0.475, 0.035, 0.295$$",283.409036,11.987233,1
996,997,25.515185,45.999078,126.878848,0,91.994688,2 (Urgent),0 (Normal),"$$17.31, 46.91, 25.75, 34.86$$","$$1.96, 3.52, 2.27, 4.12$$","$$13.91, 14.52, 11.32, 14.42$$","$$5.17, 3.66, 2.49, 3.66$$","$$0.248, 0.235, 0.382, 0.134$$","$$0.445, 0.385, 0.144, 0.027$$","$$0.042, 0.396, 0.135, 0.427$$",96.472564,10.716784,1
997,998,60.065660,24.588106,385.678833,1,80.709891,1 (Medium),0 (Normal),"$$2.46, 13.07, 10.68, 41.93$$","$$4.80, 1.77, 1.71, 1.42$$","$$7.84, 23.46, 5.98, 5.55$$","$$4.75, 4.11, 2.74, 1.38$$","$$0.287, 0.107, 0.453, 0.153$$","$$0.070, 0.331, 0.384, 0.215$$","$$0.180, 0.293, 0.456, 0.071$$",457.913664,1.681974,0
998,999,81.646419,41.185238,373.628656,3,32.678650,1 (Medium),0 (Normal),"$$44.09, 30.88, 28.60, 30.40$$","$$3.06, 3.37, 1.56, 3.36$$","$$13.55, 12.18, 22.20, 10.50$$","$$1.55, 4.53, 2.73, 5.81$$","$$0.284, 0.223, 0.177, 0.317$$","$$0.407, 0.213, 0.027, 0.354$$","$$0.176, 0.326, 0.100, 0.398$$",506.809151,9.847577,0
